In [1]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [1]:
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

NameError: name 'documents_raw' is not defined

In [2]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    return hash_hex[:8]

for doc in documents:
    doc['id'] = generate_document_id(doc)

In [3]:
from collections import defaultdict

In [4]:
hashes = defaultdict(list)
for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)


In [1]:
assert len(hashes) == len(documents)

NameError: name 'hashes' is not defined

In [12]:
import json
with open('data/documents-with-ids.json','wt') as f:
    json.dump(documents, f, indent=2)

In [18]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. 
The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [14]:
from openai import OpenAI
client = OpenAI()

URL('https://api.openai.com/v1/')

In [16]:
prompt = prompt_template.format(**doc)
print(prompt)

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: Module 6: Best practices
question: How to destroy infrastructure created via GitHub Actions
answer: Problem description
Infrastructure created in AWS with CD-Deploy Action needs to be destroyed
Solution description
From local:
terraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure
terraform destroy --var-file vars/prod.tfvars
Added by Erick Calderin

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [21]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='o4-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [23]:
from tqdm.auto import tqdm

In [24]:
results = {}

for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue
    questions= generate_questions(doc)
    results[doc_id] = questions

In [25]:
import pickle

In [26]:
with open('data/results.bin', 'rb') as f_in:
    results = pickle.load(f_in)
    

In [28]:
list(results.items())[:3]

[('c02e79ef',
  '["When does the course begin?", "How can I get the course schedule?", "What is the link for course registration?", "How can I receive course announcements?", "Where do I join the Slack channel?"]'),
 ('1f6520ca',
  '["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'),
 ('7842b56a',
  '[\n  "Can I enroll in the course after it starts?",\n  "Is late registration possible?",\n  "Am I eligible to submit homework if I join late?",\n  "Are there deadlines for final projects if I join late?",\n  "Can I submit all assignments at the end of the course if I start late?"\n]')]

In [ ]:
parsed_results ={}

In [29]:
for doc_id, json_questions in results.items():
    if doc_id in parsed_results:
        continue
    try:
        parsed_results[doc_id] = json.loads(json_questions)
    except:
        print(f"Problem with doc with id: {doc_id}")

Problem with doc with id: 58c9f99f


In [37]:
#manual cleaning faulty docs
parsed_results['58c9f99f'] =[
    r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
    "What should I do if I encounter an invalid mode error in Docker on Windows?",
    "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
    "Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
    r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [44]:
doc_index = {d['id']:d for d in documents}

In [47]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course,doc_id))

In [49]:
import pandas as pd

In [51]:
df = pd.DataFrame(final_results, columns=['query', 'course', 'document'])

In [53]:
df.head(10)

,query,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
5,Where can I find the prerequisites for this co...,data-engineering-zoomcamp,1f6520ca
6,How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
7,Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
8,What are the requirements for joining this cou...,data-engineering-zoomcamp,1f6520ca
9,Where is the list of prerequisites for the cou...,data-engineering-zoomcamp,1f6520ca


In [54]:
df.to_csv('data/ground-truth-data.csv', index=False)